In [41]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [42]:
# taken from fragile families assignment
def factorize(df):
    """Convert features of type 'object', e.g. 'string', to categorical
    variables or factors."""
    for col in df.columns:
        if df.loc[:,col].dtype == object:
            factors, values = pd.factorize(df[col])
            df.loc[:,col] = factors
    return df

In [43]:
train = pd.read_csv('train_2010_2017.csv')
train = factorize(train)
test = pd.read_csv('test_2018.csv')
test = factorize(test)

In [44]:
train_Y = train['Outcome']
train_X = train.drop(labels=['Outcome'], axis=1)

test_Y = test['Outcome']
test_X = test.drop(labels=['Outcome'], axis=1)

In [45]:
lr = LogisticRegression()
lr.fit(train_X, train_Y)
predicted_Y = lr.predict(test_X)
print metrics.accuracy_score(test_Y, predicted_Y)
print metrics.recall_score(test_Y, predicted_Y)

0.9375
0.96875
